# Helpdesk Hard

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@localhost/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 100)

 ····


In [2]:
shift = pd.read_sql_table('Shift', engine)
staff = pd.read_sql_table('Staff', engine)
issue = pd.read_sql_table('Issue', engine)
shift_type = pd.read_sql_table('Shift_type', engine)
level = pd.read_sql_table('Level', engine)
customer = pd.read_sql_table('Customer', engine)
caller = pd.read_sql_table('Caller', engine)

## 11.
Show the manager and number of calls received for each hour of the day on 2017-08-12

```
+---------+---------------+----+
| Manager | Hr            | cc |
+---------+---------------+----+
| LB1     | 2017-08-12 08 |  6 |
| LB1     | 2017-08-12 09 | 16 |
| LB1     | 2017-08-12 10 | 11 |
| LB1     | 2017-08-12 11 |  6 |
| LB1     | 2017-08-12 12 |  8 |
| LB1     | 2017-08-12 13 |  4 |
| AE1     | 2017-08-12 14 | 12 |
| AE1     | 2017-08-12 15 |  8 |
| AE1     | 2017-08-12 16 |  8 |
| AE1     | 2017-08-12 17 |  7 |
| AE1     | 2017-08-12 19 |  5 |
+---------+---------------+----+
```

In [19]:
(issue.assign(date_call=issue['Call_date'].dt.date)
 .assign(hr=issue['Call_date'].dt.strftime('%Y-%m-%d %H'))
 .astype({'date_call': 'datetime64'})
 .query('date_call=="2017-08-12"')
 .merge(shift, left_on=['Taken_by', 'date_call'], 
        right_on=['Operator', 'Shift_date'])
 [['Manager', 'hr', 'Call_ref']]
 .groupby(['Manager', 'hr'])
 .count().reset_index()
 .sort_values('hr'))

,Manager,hr,Call_ref
5,LB1,2017-08-12 08,6
6,LB1,2017-08-12 09,16
7,LB1,2017-08-12 10,11
8,LB1,2017-08-12 11,6
9,LB1,2017-08-12 12,8
10,LB1,2017-08-12 13,4
0,AE1,2017-08-12 14,12
1,AE1,2017-08-12 15,8
2,AE1,2017-08-12 16,8
3,AE1,2017-08-12 17,7


## 12.
**80/20 rule. It is said that 80% of the calls are generated by 20% of the callers. Is this true? What percentage of calls are generated by the most active 20% of callers.**

Note - Andrew has not managed to do this in one query - but he believes it is possible.

```
+---------+
| t20pc   |
+---------+
| 32.2581 |
+---------+
```

## 13.
**Annoying customers. Customers who call in the last five minutes of a shift are annoying. Find the most active customer who has never been annoying.**

```
+--------------+------+
| Company_name | abna |
+--------------+------+
| High and Co. |   20 |
+--------------+------+
```

## 14.
**Maximal usage. If every caller registered with a customer makes a call in one day then that customer has "maximal usage" of the service. List the maximal customers for 2017-08-13.**

```
+-------------------+--------------+-------------+
| company_name      | caller_count | issue_count |
+-------------------+--------------+-------------+
| Askew Inc.        |            2 |           2 |
| Bai Services      |            2 |           2 |
| Dasher Services   |            3 |           3 |
| High and Co.      |            5 |           5 |
| Lady Retail       |            4 |           4 |
| Packman Shipping  |            3 |           3 |
| Pitiable Shipping |            2 |           2 |
| Whale Shipping    |            2 |           2 |
+-------------------+--------------+-------------+
```

## 15.
**Consecutive calls occur when an operator deals with two callers within 10 minutes. Find the longest sequence of consecutive calls – give the name of the operator and the first and last call date in the sequence.**

```
+----------+---------------------+---------------------+-------+
| taken_by | first_call          | last_call           | calls |
+----------+---------------------+---------------------+-------+
| AB1      | 2017-08-14 09:06:00 | 2017-08-14 10:17:00 |    24 |
+----------+---------------------+---------------------+-------+
```